In [4]:
# Try "pip install odfpy" and "pip install openpyxl" if error raised related to ".xlsx" files

import pandas as pd 
import numpy as numpy

def load_data(em_data = True, veh_data = True, nox_data = True, pm_data = True):
    args = []
    # Emission Data (CO2)
    EU_em_df = pd.read_excel('Data/Table_11.5_Carbon_Dioxide_Emissions_From_Energy_Consumption__Transportation_Sector.xlsx',engine='openpyxl')
    EU_em_annual_df = pd.read_csv('Data/EU_annual_carbon_emission_by_road_transportation.csv',)
    UK_em_df = pd.read_excel('Data/uk-env0201.ods', nrows = 16)
    US_em_df = pd.read_excel('Data/US_em.xlsx',engine='openpyxl')
    
    # Emissions data Processing
    EU_em_df = EU_em_df.set_index('Month')
    UK_em_df = UK_em_df.transpose()
    
    # Electric Vehicles
    EU_veh_df = pd.read_csv('Data/new-electric-vehicles-in-eu-1.csv')
    UK_veh_df = pd.read_excel('Data/veh0203.ods', nrows= 76)
    US_veh_df = pd.read_excel('Data/10354_epact_vehicle_history_5-21-21.xlsx',engine='openpyxl')
    args += [EU_em_df, EU_em_annual_df, UK_em_df, US_em_df, EU_veh_df,UK_veh_df, US_veh_df]

    # Secondary Data 
        # Air pollution
            # UK 1970-2019/2022
    UK_nox_annual_df = pd.read_csv('Data/Figure06_NOx_time_series.csv')
    UK_pm_all_annual_df = pd.read_csv('Data/Figure03_PM_time_series.csv')
            
            # USA Since 1970-2020, plz Access through column id(numbers)
    USA_nox_annual_df = pd.read_csv('Data/US_nox_em_time_series.csv') # National Emissions Totals (thousands of tons)
    USA_pm_10_annual_df = pd.read_csv('Data/US_pm10_year.csv')        #
    USA_pm_2_5_annual_df = pd.read_csv('Data/US_pm2_5_year.csv')
    
    USA_pm_2_5_annual_df = USA_pm_2_5_annual_df.transpose()
    USA_pm_10_annual_df = USA_pm_10_annual_df.transpose()
            
            # OCED DATA
    OCED_PM10_df = pd.read_excel('Data/PM10_ROAD_OCED_WORLD_DATA.xlsx',engine='openpyxl')
    OCED_NOX_df = pd.read_excel('Data/NOX_ROAD_OCED_WORLD_DATA.xlsx',engine='openpyxl')
    OCED_PM2_5_df = pd.read_excel('Data/PM2_5_ROAD_OCED_WORLD_DATA.xlsx',engine='openpyxl')
    args += [UK_nox_annual_df,UK_pm_all_annual_df,USA_pm_10_annual_df,USA_pm_2_5_annual_df,USA_nox_annual_df,OCED_PM10_df,OCED_NOX_df,OCED_PM2_5_df]
    return args

EU_em_df, EU_em_annual_df, UK_em_df, US_em_df, EU_veh_df,UK_veh_df, US_veh_df, UK_nox_annual_df,UK_pm_all_annual_df,USA_pm_10_annual_df,USA_pm_2_5_annual_df,USA_nox_annual_df,OCED_PM10_df,OCED_NOX_df,OCED_PM2_5_df = load_data()

# Eric: Simpler ways to read the data
# calling head() method  
# storing in new variable 
data_top = OCED_NOX_df
    
# display 
data_top 

#EU_em_df, UK_em_df, US_em_df

C:\Apps\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Australia,549.156,539.232,580.854,588.587,596.750,624.957,633.879,642.463,627.553,...,628.437,630.849,622.656,623.603,625.602,648.935,670.229,693.565,693.805,651.806
1,Austria,131.858,140.863,136.841,133.585,128.986,127.164,145.665,131.425,146.159,...,140.279,132.955,128.200,128.653,124.115,119.344,112.539,103.602,95.319,88.761
2,Belgium,230.964,231.581,239.363,235.217,225.982,215.270,207.132,198.666,195.381,...,134.059,126.231,119.361,112.641,108.246,107.413,100.740,92.579,84.595,78.041
3,Canada,1245.247,1187.547,1203.455,1239.369,1318.584,1353.281,1393.927,1474.823,1502.152,...,1015.033,983.469,916.365,874.483,808.944,757.804,713.621,747.369,780.810,786.101
4,Chile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,65.399,66.761,75.581,80.366,81.937,60.575,54.173,56.035,54.500,NaN
5,Costa Rica,16.637,17.089,19.145,21.251,23.834,25.138,25.934,26.982,29.669,...,44.340,45.400,46.718,47.399,48.399,51.591,55.102,56.134,NaN,NaN
6,Czech Republic,199.859,182.674,183.067,170.362,170.983,153.170,143.934,134.051,134.296,...,90.293,87.753,85.023,81.244,80.684,78.067,75.827,76.045,72.760,68.911
7,Denmark,159.293,165.659,162.864,159.379,157.964,154.823,150.296,144.761,138.729,...,90.850,85.879,79.332,76.330,70.220,68.103,65.590,61.643,57.887,51.564
8,Estonia,38.736,36.727,20.541,20.930,23.005,20.265,21.948,22.485,23.472,...,15.917,14.851,14.255,12.912,12.870,12.369,11.158,10.791,10.416,9.607
9,Finland,166.643,165.273,164.010,163.749,161.280,155.356,152.529,151.381,143.973,...,74.155,70.145,65.327,63.012,59.416,55.731,51.760,48.039,45.392,42.229
